# LightGlue Demo
In this notebook we match two pairs of images using LightGlue with early stopping and point pruning.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
# !unzip "gallery.zip" -d "gallery"

In [5]:
# If we are on colab: this clones the repo and installs the dependencies
from pathlib import Path

if Path.cwd().name != "LightGlue":
    !git clone --quiet https://github.com/cvg/LightGlue/
    %cd LightGlue
    !pip install --progress-bar off --quiet -e .


/content/drive/MyDrive/LightGlue
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for lightglue (pyproject.toml) ... done


In [30]:
from lightglue import LightGlue, SuperPoint, DISK
from lightglue.utils import load_image, rbd
from lightglue import viz2d
import torch
import numpy as np
from lightglue import match_pair
from pathlib import Path
import matplotlib.pyplot as plt
import os
import cv2
import sys

In [26]:
torch.set_grad_enabled(False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 'mps', 'cpu'

extractor = SuperPoint(max_num_keypoints=2048).eval().to(device)  # load the extractor
matcher = LightGlue(features="superpoint").eval().to(device)
print(device)

Downloading: "https://github.com/cvg/LightGlue/releases/download/v0.1_arxiv/superpoint_v1.pth" to /root/.cache/torch/hub/checkpoints/superpoint_v1.pth
100%|██████████| 4.96M/4.96M [00:00<00:00, 331MB/s]
Downloading: "https://github.com/cvg/LightGlue/releases/download/v0.1_arxiv/superpoint_lightglue.pth" to /root/.cache/torch/hub/checkpoints/superpoint_lightglue_v0-1_arxiv.pth
100%|██████████| 45.3M/45.3M [00:01<00:00, 40.8MB/s]


cuda


In [ ]:
query_path = Path("gallery/gallery/query_boxed")
gallery_path = Path("gallery/gallery/gallery")

file_list = os.listdir("gallery/gallery/query_boxed/")
file_list.sort(key=lambda x:x.split(".")[0])

for i, file_name in enumerate(file_list):
    query_image = load_image(query_path / file_name)
    # cv2.imread(str(query_path / file_name))
    file_list = os.listdir(str(gallery_path))
    scores = []
    for i, file_name1 in enumerate(file_list):
        percentage = round(i / len(file_list) * 100)
        print("\r文件处理进度: {}%: ".format(percentage), "▓" * (percentage // 2), end="")
        sys.stdout.flush()
        gallery_image = load_image(gallery_path / file_name1)
        _, _, matches = match_pair(extractor, matcher, query_image.cuda(), gallery_image.cuda())
        if len(matches['scores']) > 0:
            score = np.mean(np.array(matches['scores']))
        else: score=0
        scores.append((file_name1.split(".")[0], score))
    scores.sort(key=lambda x: x[1], reverse=True)
    with open('ORB_deep.txt', 'a', encoding='utf-8') as file:
        file.write("Q" + str(int(file_name.split('.')[0])+1) + ": ")
        order = ""
        for num, _ in scores:
            order = order + num + " "
        file.write(order)
        file.write("\n")

文件处理进度: 49%:  ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓